# Test monitor

In [5]:
import threading

class CarParkingMonitor :
    def __init__(self, capacity) :
        self.capacity = capacity
        #self.free = threading.Semaphore(capacity)
        #self.toEmpty = threading.Semaphore(0)
        self.freePlaces = capacity
        self.useLock = threading.Lock()
        self.useAvailable = threading.Condition(self.useLock)
    
    def takePlace(self) :
        with self.useLock :
            while self.freePlaces <=0 :
                self.useAvailable.wait()
            self.freePlaces=self.freePlaces-1
            self.useAvailable.notifyAll()
        
    def freePlace(self) :
        with self.useLock :
            while self.freePlaces >= self.capacity :
                self.useAvailable.wait()
            self.freePlaces=self.freePlaces+1
            self.useAvailable.notifyAll()

monitor = CarParkingMonitor(3)
printLock = threading.Lock()

def whoArrive(i) :
    global monitor
    global printLock
    printLock.acquire()
    if monitor.capacity==0 : 
        print(i,"is waiting to park")
    printLock.release()
    monitor.takePlace()
    printLock.acquire()
    print(i,"parked")
    printLock.release()
    
    
def whoLeave(i) :
    global monitor
    global printLock
    printLock.acquire()
    if monitor.capacity!=0 : 
        print(i,"is waiting to leave")
    printLock.release()
    monitor.freePlace()
    printLock.acquire()
    print(i,"leaved")
    printLock.release()
    
for i in range(1,10) :
    t1 = threading.Thread(target = whoArrive, args=(i,))
    t1.start()
    
for i in range(1,10) :
    t2 = threading.Thread(target = whoLeave, args=(10+i,))
    t2.start()
    

1 parked
2 parked
3 parked
11 is waiting to leave
12 is waiting to leave
11 leaved
8 parked
12 leaved
6 parked
13 is waiting to leave
13 leaved
14 is waiting to leave
14 leaved
5 parked
4 parked
15 is waiting to leave
15 leaved
17 is waiting to leave
17 leaved
16 is waiting to leave
7 parked
18 is waiting to leave
18 leaved
19 is waiting to leave
16 leaved
9 parked
19 leaved
